In [1]:
%load_ext zipline

In [2]:
from zipline.api import *
from zipline.algorithm import *
import zipline
from plotly.offline import init_notebook_mode , plot , iplot , download_plotlyjs
import pandas as pd
import numpy as np
import plotly.plotly as py
import cufflinks as cf
from datetime import datetime
init_notebook_mode(connected=True)
cf.go_offline()

ImportError: No module named 'plotly'

In [5]:
def initialize(context):
    context.security = [symbol('AAPL'),symbol('AMZN'),symbol('GOOGL'),symbol('EBAY'),symbol('ACTG')]
#     context.security = [symbol("AAPL")]
    schedule_function(rsi_handling,
                      date_rules.every_day(),
                      time_rules.market_open(hours = 1)
                     )
    
def cal_rsi(df):   
    win = df[df > 0] 
    lose = df[df < 0] 
    up = win.mean() if len(win) > 0 else 0 
    down = abs(lose.mean()) if len(lose) > 0 else 0 
    return 100 * up / (up + down) 

def rsi_handling(context , data):
    lo = 9
    sel = 80
    buy = 25
    for st in context.security :
        df = data.history(st ,
                            'price' , (lo+1) ,
                            '1d' )
        df = df.diff()
        ''' RSI Algorithms '''    
        ct = 0
        rsil = df.rolling(lo).agg(cal_rsi)
        rsil = rsil.fillna(0)
        
        open_orders = get_open_orders()
        
        if rsil[-1] <= buy :
            if st not in open_orders:
                order_target_percent(st , 0.2 )
                print("buy trade at {}".format(context.get_datetime()))
                ct+=10
        elif rsil[-1] >= sel :
            if st in open_orders:
                order_target_percent(st , -0.2 )
                print("sell trade at {}".format(context.get_datetime()))
                ct -=10
    ''' END RSI Algorithms '''
    record(ct = ct, rsil = rsil[-1])

def analyze(context , pref ):
    buy = 20
    sel = 80
    pref['gross_leverage'].iplot(title='gross lev')
#     pref[['rsil','ct']].iplot(hline=[sel,buy])
    pref[['benchmark_period_return','algorithm_period_return']].iplot(title='strategy return')
    pref['returns'].iplot(title='return')


In [6]:
%zipline --bundle quantopian-quandl --start 2015-1-4 --end 2016-1-4 -o backtest.pickle

buy trade at 2015-01-05 21:00:00+00:00
buy trade at 2015-01-07 21:00:00+00:00
buy trade at 2015-01-08 21:00:00+00:00
buy trade at 2015-01-09 21:00:00+00:00
buy trade at 2015-01-12 21:00:00+00:00
buy trade at 2015-02-02 21:00:00+00:00
buy trade at 2015-02-20 21:00:00+00:00
buy trade at 2015-02-23 21:00:00+00:00
buy trade at 2015-03-09 20:00:00+00:00
buy trade at 2015-03-09 20:00:00+00:00
buy trade at 2015-03-10 20:00:00+00:00
buy trade at 2015-03-11 20:00:00+00:00
buy trade at 2015-04-13 20:00:00+00:00
buy trade at 2015-04-14 20:00:00+00:00
buy trade at 2015-04-15 20:00:00+00:00
buy trade at 2015-04-17 20:00:00+00:00
buy trade at 2015-04-17 20:00:00+00:00
buy trade at 2015-04-20 20:00:00+00:00
buy trade at 2015-04-21 20:00:00+00:00
buy trade at 2015-04-22 20:00:00+00:00
buy trade at 2015-06-09 20:00:00+00:00
buy trade at 2015-06-18 20:00:00+00:00
buy trade at 2015-07-30 20:00:00+00:00
buy trade at 2015-07-31 20:00:00+00:00
buy trade at 2015-08-03 20:00:00+00:00
buy trade at 2015-08-04 2

,algo_volatility,algorithm_period_return,alpha,benchmark_period_return,benchmark_volatility,beta,capital_used,ct,ending_cash,ending_exposure,...,short_exposure,short_value,shorts_count,sortino,starting_cash,starting_exposure,starting_value,trading_days,transactions,treasury_period_return
2015-01-05 21:00:00+00:00,NaN,0.000000,NaN,-0.004916,NaN,NaN,0.000000e+00,10,1.000000e+07,0.000000e+00,...,0.0,0.0,0,NaN,1.000000e+07,0.000000e+00,0.000000e+00,1,[],0.0
2015-01-06 21:00:00+00:00,0.000976,-0.000087,NaN,-0.009808,0.000000,NaN,-1.553478e+06,0,8.446522e+06,1.552608e+06,...,0.0,0.0,0,-11.224972,1.000000e+07,0.000000e+00,0.000000e+00,2,"[{'commission': None, 'sid': Equity(42 [ACTG])...",0.0
2015-01-07 21:00:00+00:00,0.007069,0.000637,NaN,-0.014676,0.000000,NaN,-4.413519e+05,10,8.005170e+06,2.001200e+06,...,0.0,0.0,0,67.121379,8.446522e+06,1.552608e+06,1.552608e+06,3,"[{'commission': None, 'sid': Equity(42 [ACTG])...",0.0
2015-01-08 21:00:00+00:00,0.017879,-0.001284,NaN,-0.019519,0.000000,NaN,-6.607702e+01,10,8.005104e+06,1.982059e+06,...,0.0,0.0,0,-5.297565,8.005170e+06,2.001200e+06,2.001200e+06,4,"[{'commission': None, 'sid': Equity(42 [ACTG])...",0.0
2015-01-09 21:00:00+00:00,0.017041,-0.002605,NaN,-0.024339,0.000000,NaN,-1.527697e+04,0,7.989827e+06,1.984121e+06,...,0.0,0.0,0,-7.928829,8.005104e+06,1.982059e+06,1.982059e+06,5,"[{'commission': None, 'sid': Equity(42 [ACTG])...",0.0
2015-01-12 21:00:00+00:00,0.030445,-0.007181,NaN,-0.029136,0.000000,NaN,-1.980784e+06,10,6.009043e+06,3.919147e+06,...,0.0,0.0,0,-9.057598,7.989827e+06,1.984121e+06,1.984121e+06,6,"[{'commission': None, 'sid': Equity(1245 [GOOG...",0.0
2015-01-13 21:00:00+00:00,0.032644,-0.005538,NaN,-0.033908,0.000000,NaN,-4.624687e+04,0,5.962796e+06,3.981828e+06,...,0.0,0.0,0,-6.456062,6.009043e+06,3.919147e+06,3.919147e+06,7,"[{'commission': None, 'sid': Equity(42 [ACTG])...",0.0
2015-01-14 21:00:00+00:00,0.036149,-0.009838,NaN,-0.038658,0.000000,NaN,0.000000e+00,0,5.962796e+06,3.938820e+06,...,0.0,0.0,0,-8.234134,5.962796e+06,3.981828e+06,3.981828e+06,8,[],0.0
2015-01-15 21:00:00+00:00,0.033815,-0.011099,NaN,-0.043384,0.000000,NaN,0.000000e+00,0,5.962796e+06,3.926218e+06,...,0.0,0.0,0,-8.612034,5.962796e+06,3.938820e+06,3.938820e+06,9,[],0.0
2015-01-16 21:00:00+00:00,0.066384,-0.000852,NaN,-0.048086,0.000000,NaN,0.000000e+00,0,5.962796e+06,4.028688e+06,...,0.0,0.0,0,-0.567541,5.962796e+06,3.926218e+06,3.926218e+06,10,[],0.0


limit_output extension: Maximum message size of 10000 exceeded with 41815 characters